[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rHmUqB0QHmSkRd7Wh7tzZFN5vF5j8v1W)

# Setting Up

In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!git clone git@github.com:geetu040/llm-proj.git
!mv llm-proj/* .
!rm -rf llm-proj
!pip install -r requirements.txt
!mkdir processed_data
!python prepare_excel_data.py --path "data/NUST Bank-Product-Knowledge.xlsx" --output "processed_data/excel_output.json"
!python prepare_json_data.py --path "data/funds_transfer_app_features_faq (1).json" --output "processed_data/json_output.json"

Cloning into 'llm-proj'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 80 (delta 38), reused 64 (delta 22), pack-reused 0 (from 0)
Receiving objects: 100% (80/80), 305.29 KiB | 8.25 MiB/s, done.
Resolving deltas: 100% (38/38), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 12.6 MB/s eta 0:00:00
Extracted QA pairs saved to processed_data/excel_output.json
Extracted QA pairs saved to processed_data/json_output.json


In [ ]:
import os
import json
import torch
import time
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    set_seed
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from functools import partial
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

# Initialize Model

In [ ]:
max_seq_length = 5020
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_name = 'google/flan-t5-large'

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Data Preparation

In [ ]:
data = {}
with open("processed_data/excel_output.json", 'rb') as f:
    data.update(json.load(f))
with open("processed_data/json_output.json", 'rb') as f:
    data.update(json.load(f))

dataset = Dataset.from_dict({
    "question": list(data.keys()),
    "answer": list(data.values()),
})

dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 345
})

In [ ]:
dataset[0], dataset[150]

({'question': '(Main) Liability Products & Services',
  'answer': 'NUST Asaan Account (NAA). Little Champs Account. NUST Sahar Accounts. NUST Waqaar Account. PakWatan Remittance Account. NUST Home Remittance. RDA Digital Customer Onboarding. Current Deposit Account (CDA). Value Plus Current Account (Individual) (VPCA). Value Plus Business Account (VP-BA). Value Premium Business Account (VPBA). NUST Maximiser Account. NUST Special Deposit Account (NSDA). Profit And Loss Sharing Account (PLS). NUST Asaan Digital Account. NUST Asaan Digital Remittance Account. NUST Freelancer Digital Account.'},
 {'question': '(NMC) 4.Can overseas Pakistanis obtain Credit Card facility from NUST?',
  'answer': 'Yes. The individuals maintaining FC account with NUST, Remittances/ Roshan Digital Account holders/. Holders of Naya Pakistan Certificates can apply for NUST MasterCard Credit Cards on lien basis.. Note: Facility can only be allowed in case of FC joint account maintaining with NUST (one person shou

In [ ]:
prompt_template = """You are a helpful, caring and knowledgeable assistant for a bank customer service.

Question: {0}

Answer: {1}"""

def formatting_prompt(examples):
    inputs = examples["question"]
    outputs = examples["answer"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = prompt_template.format(input_, output).strip()
        texts.append(text)
    return {'texts': texts}

dataset = dataset.map(
    formatting_prompt,
    batched=True,
)

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [ ]:
def generate_response(question):
    prompt = prompt_template.format(question, "").strip()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=max_seq_length,
        pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
def process_batch(batch):
    return tokenizer(batch['texts'], padding=True)

dataset = dataset.map(
    process_batch,
    batched=True,
    remove_columns=['question', 'answer', 'texts'],
)

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.shuffle()

# Fine-Tuning

In [ ]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=[
        'q',
        'k',
        'v',
        'o',
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)
model.gradient_checkpointing_enable()

model = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir = "outputs/",
    warmup_steps=1,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=1e-3,
    optim="paged_adamw_8bit",
    logging_steps=3,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    eval_strategy="no",
    do_eval=False,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
    dataloader_pin_memory=False,
    save_total_limit=3,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
)

trainer.train()

Truncating train dataset:   0%|          | 0/345 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
3,36.640900
6,28.808300
9,19.772900
12,14.139400
15,22.487500
18,17.323100
21,7.255000
24,7.235700
27,6.181800
30,5.554800


TrainOutput(global_step=87, training_loss=6.680728630772952, metrics={'train_runtime': 325.0033, 'train_samples_per_second': 1.062, 'train_steps_per_second': 0.268, 'total_flos': 692558022758400.0, 'train_loss': 6.680728630772952})

In [ ]:
model = trainer.model
model = model.merge_and_unload()

In [ ]:
model.save_pretrained("tuned_model")
tokenizer.save_pretrained("tuned_model")

('tuned_model/tokenizer_config.json',
 'tuned_model/special_tokens_map.json',
 'tuned_model/spiece.model',
 'tuned_model/added_tokens.json',
 'tuned_model/tokenizer.json')